In [1]:
from google.colab import drive
drive.mount('/content/drive' , force_remount = 'True')
#!ls "/content/drive/My Drive"

#module imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#SCIPY
from scipy.stats import norm


#SKLEARN
from sklearn.model_selection import train_test_split

#
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

#
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Function to simulate a Geometric Brownian Motion (GBM)
def geometric_brownian_motion(S0, mu, sigma, T, dt):
    N = int(T / dt)  # Number of time steps
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size=N)
    ww = W
    W = np.cumsum(W) * np.sqrt(dt)  # Brownian motion
    S = S0 * np.exp((mu - 0.5 * sigma**2) * t + sigma * W)  # GBM formula

    # Generate simulations
    simulations=np.zeros(len(S))
    simulations[0] = S0
    for i in range(1, N):
        dW = ww[i] * np.sqrt(dt)  # Brownian motion increment
        simulations[i] = simulations[i-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * dW)

    return t, S, simulations

# Parameters
S0 = 100          # Initial stock price
mu = 0.05         # Expected return
sigma = 0.2       # Volatility
T = 1             # Time to maturity in years
dt = 1/252        # Daily time step (assuming 252 trading days in a year)

# Simulate GBM
time, stock_prices, ss = geometric_brownian_motion(S0, mu, sigma, T, dt)

# Plot the simulated stock prices
plt.figure(figsize=(10, 6))
plt.plot(time, stock_prices , label="GBM Stock Price")
plt.plot(time[1:], ss[1:] , label="GBM Stock Price")
plt.title("Geometric Brownian Motion Simulation")
plt.xlabel("Time (years)")
plt.ylabel("Stock Price")
plt.legend()
plt.grid()
plt.show()




In [ ]:
##High risk investing : Binary option using BSM

# Function to calculate the price of a stock using the Geometric Brownian Motion model
def stock_price(S0, mu, sigma, T, dt):
    N = int(T / dt)  # Number of time steps
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size=N)
    W = np.cumsum(W) * np.sqrt(dt)  # Brownian motion
    return S0 * np.exp((mu - 0.5 * sigma**2) * t + sigma * W)

# Function to calculate the price of a digital option using the Black-Scholes model
def digital_option_price(S, K, r, sigma, T, option_type="call"):
    d2 = (np.log(S / K) + (r - 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == "call":
        return np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        return np.exp(-r * T) * norm.cdf(-d2)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

# Parameters
S0 = 100          # Initial stock price
mu = 0.05         # Expected return
sigma = 0.2       # Volatility
T = 0.5           # Time to maturity in years (6 months)
dt = 1/252        # Daily time step (assuming 252 trading days in a year)
K = 105           # Strike price for the option
r = 0.03          # Risk-free rate

# Simulate stock price for 6 months
simulated_stock_prices = stock_price(S0, mu, sigma, T, dt)
final_stock_price = simulated_stock_prices[-1]

# Price the digital call and put options
digital_call_price = digital_option_price(final_stock_price, K, r, sigma, T, option_type="call")
digital_put_price = digital_option_price(final_stock_price, K, r, sigma, T, option_type="put")

# Output results
print("Final Stock Price after 6 months:", round(final_stock_price, 2))
print("Digital Call Option Price:", round(digital_call_price, 4))
print("Digital Put Option Price:", round(digital_put_price, 4))



In [ ]:
#Conservative investor: Commodity-Linked bond (CLB)

# Function to calculate the coupon payment based on commodity price
def calculate_coupon(commodity_price, base_price, base_coupon_rate, sensitivity_factor):
    """
    Calculate the coupon payment based on commodity price.
    :param commodity_price: The price of the commodity.
    :param base_price: The base price of the commodity at issuance.
    :param base_coupon_rate: The base coupon rate at issuance.
    :param sensitivity_factor: Sensitivity of the coupon rate to commodity price changes.
    :return: The coupon payment.
    """
    # The coupon rate increases/decreases based on the commodity price change
    price_change_factor = (commodity_price / base_price) * sensitivity_factor
    adjusted_coupon_rate = base_coupon_rate + price_change_factor
    return adjusted_coupon_rate

# Function to calculate the bond price
def calculate_bond_price(face_value, coupons, yield_rate):
    """
    Calculate the bond price using present value of cash flows.
    :param face_value: The face value of the bond.
    :param coupons: The list of coupon payments for each year.
    :param yield_rate: The required yield (discount rate).
    :return: The bond price.
    """
    bond_price = 0
    for t, coupon in enumerate(coupons, start=1):
        bond_price += coupon / (1 + yield_rate)**t
    # Add the face value discounted to the present value
    bond_price += face_value / (1 + yield_rate)**len(coupons)
    return bond_price

# Parameters
face_value = 1000  # Face value of the bond (USD)
base_price = 100  # Base price of the commodity (USD)
base_coupon_rate = 0.05  # Base coupon rate (5%)
sensitivity_factor = 0.02  # Sensitivity factor to commodity price (e.g., 2% per unit increase in commodity price)
maturity_years = 5  # Maturity of the bond in years
yield_rate = 0.03  # Required yield (3%)

# Commodity prices over time (in USD) for each year
commodity_prices = [105, 110, 120, 115, 100]  # Example commodity prices for each year, historical data can be used

# Calculate coupon payments for each year
coupons = [face_value * calculate_coupon(price, base_price, base_coupon_rate, sensitivity_factor) for price in commodity_prices]

# Calculate the bond price
bond_price = calculate_bond_price(face_value, coupons, yield_rate)

# Display results
print(f"Commodity Prices: {commodity_prices}")
print(f"Coupon Payments: {coupons}")
print(f"Bond Price (Present Value): ${bond_price:.2f}")


In [ ]:
# Moderate risk investor  : Structured notes

import numpy as np
from scipy.stats import norm

# Function to calculate the price of a call option using the Black-Scholes formula
def black_scholes_call(S, K, T, r, sigma):
    """
    Calculate the price of a European call option using the Black-Scholes formula.
    :param S: Current price of the underlying asset (stock).
    :param K: Strike price of the option.
    :param T: Time to maturity in years.
    :param r: Risk-free interest rate (annualized).
    :param sigma: Volatility of the underlying asset (annualized).
    :return: Price of the call option.
    """
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate the coupon payment based on underlying asset performance
def calculate_coupon(S, K, base_coupon_rate, sensitivity_factor):
    """
    Calculate the coupon payment based on the underlying asset's price.
    :param S: Current price of the underlying asset (stock).
    :param K: Strike price of the option.
    :param base_coupon_rate: The base coupon rate (fixed) without considering the asset performance.
    :param sensitivity_factor: Sensitivity of the coupon rate to the underlying asset price change.
    :return: Coupon payment.
    """
    # The coupon payment depends on how the asset's price compares to the strike price
    price_change_factor = (S / K) * sensitivity_factor  # Adjust coupon based on asset price
    adjusted_coupon_rate = base_coupon_rate + price_change_factor
    return adjusted_coupon_rate

# Function to calculate the structured note price with principal protection and coupon payments
def structured_note_price(face_value, strike_price, S0, maturity, r, sigma, coupon_frequency, base_coupon_rate, sensitivity_factor):
    """
    Calculate the price of a structured note with principal protection and periodic coupon payments.
    :param face_value: Face value of the structured note (USD).
    :param strike_price: Strike price of the embedded call option.
    :param S0: Initial price of the underlying asset (stock).
    :param maturity: Time to maturity in years.
    :param r: Risk-free interest rate (annualized).
    :param sigma: Volatility of the underlying asset (annualized).
    :param coupon_frequency: Number of coupon payments per year (e.g., 1 for annual).
    :param base_coupon_rate: The base coupon rate (fixed).
    :param sensitivity_factor: Sensitivity of the coupon rate to the asset price change.
    :return: Price of the structured note.
    """
    # Price of the call option embedded in the note (Black-Scholes model)
    call_option_price = black_scholes_call(S0, strike_price, maturity, r, sigma)

    # Calculate coupon payments over the life of the note
    coupons = []
    for t in range(1, int(maturity * coupon_frequency) + 1):
        # Simulate the price of the underlying asset at each coupon date (for simplicity)
        S_t = S0 * np.exp((r - 0.5 * sigma ** 2) * (t / coupon_frequency) + sigma * np.sqrt(t / coupon_frequency) * np.random.normal())
        coupon = face_value * calculate_coupon(S_t, strike_price, base_coupon_rate, sensitivity_factor)
        coupons.append(coupon)

    # The structured note is the sum of discounted coupons and the discounted face value (principal protection)
    coupon_pv = sum([coupon / (1 + r/coupon_frequency)**(i + 1) for i, coupon in enumerate(coupons)])
    principal_pv = face_value / (1 + r/coupon_frequency)**(maturity * coupon_frequency)

    # Price of the structured note is the present value of coupons + present value of principal
    note_price = coupon_pv + principal_pv + call_option_price
    return note_price

# Parameters
face_value = 1000  # Face value of the structured note (USD)
strike_price = 1100  # Strike price of the embedded call option (USD)
S0 = 1000  # Initial price of the underlying asset (stock) (USD)
maturity = 5  # Time to maturity (years)
r = 0.05  # Risk-free interest rate (5%)
sigma = 0.2  # Volatility of the underlying asset (20%)
coupon_frequency = 1  # Coupon payments per year (1 = annual, 2 = semi-annual, etc.)
base_coupon_rate = 0.05  # Base coupon rate (5%)
sensitivity_factor = 0.02  # Sensitivity factor for coupon rate change

# Calculate the price of the structured note
note_price = structured_note_price(face_value, strike_price, S0, maturity, r, sigma, coupon_frequency, base_coupon_rate, sensitivity_factor)

# Display the result
print(f"Price of the Structured Note with Coupons: ${note_price:.2f}")




In [ ]:
##Diversification

# Sample data: Price of underlying assets in the ETF and their respective quantities
asset_prices = [100, 200, 50, 30]  # Prices of the assets in the ETF
asset_quantities = [10, 5, 20, 15]  # Quantities of each asset in the ETF
shares_outstanding = 1000  # Total number of ETF shares outstanding

# Calculate the total value of the ETF's assets
total_asset_value = sum(price * quantity for price, quantity in zip(asset_prices, asset_quantities))

# Calculate the Net Asset Value (NAV) per share
nav_per_share = total_asset_value / shares_outstanding

# Print the results
print(f"Total Value of ETF Assets: ${total_asset_value}")
print(f"Net Asset Value (NAV) per Share: ${nav_per_share:.2f}")
